在一个 106 x 106 的网格中，每个网格上方格的坐标为 (x, y) 。

现在从源方格 source = [sx, sy] 开始出发，意图赶往目标方格 target = [tx, ty] 。数组 blocked 是封锁的方格列表，其中每个 blocked[i] = [xi, yi] 表示坐标为 (xi, yi) 的方格是禁止通行的。

每次移动，都可以走到网格中在四个方向上相邻的方格，只要该方格 不 在给出的封锁列表 blocked 上。同时，不允许走出网格。

只有在可以通过一系列的移动从源方格 source 到达目标方格 target 时才返回 true。否则，返回 false。


示例 1：

输入：blocked = [[0,1],[1,0]], source = [0,0], target = [0,2]
输出：false
解释：
从源方格无法到达目标方格，因为我们无法在网格中移动。
无法向北或者向东移动是因为方格禁止通行。
无法向南或者向西移动是因为不能走出网格。

示例 2：
输入：blocked = [], source = [0,0], target = [999999,999999]
输出：true
解释：
因为没有方格被封锁，所以一定可以到达目标方格。


提示：

0 <= blocked.length <= 200
blocked[i].length == 2
0 <= xi, yi < 106
source.length == target.length == 2
0 <= sx, sy, tx, ty < 106
source != target
题目数据保证 source 和 target 不在封锁列表内

In [ ]:
from collections import deque


def f(blocked: list, source: list, target: list):
    blocked = set(map(tuple, blocked))
    blockThr = (len(blocked) - 1) * len(blocked) // 2 # 不包含blocked边界

    def bfs(r, c, target):
        q = deque([(r, c)])
        vis = set(q)
        while q:
            r, c = q.popleft()
            for nr, nc in ((r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)):
                if 0 <= nr < 10**6 and 0 <= nc < 10**6 and (nei := (nr, nc)) not in vis and nei not in blocked:
                    if nei == tuple(target):
                        return True
                    vis.add(nei)
                    if len(vis) > blockThr:
                        return True
                    q.append(nei)
        return False

    return bfs(*source, target) and bfs(*target, source)


f([], [0, 1], [20, 9])
# f([[0, 1], [1, 0]], [0, 0], [0, 2])
f([[10, 9], [9, 10], [10, 11], [11, 10]], [0, 0], [10, 10])


In [35]:
from collections import deque


def f(blocked: list, source: list, target: list):
    blockThr = (len(blocked) + 1) * len(blocked) // 2 # 包含blocked边界

    def bfs(r, c, target):
        q = deque([(r, c)])
        vis = set(map(tuple, blocked + [[r, c]]))
        while q:
            r, c = q.popleft()
            for nr, nc in ((r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)):
                if 0 <= nr < 10**6 and 0 <= nc < 10**6 and (nei := (nr, nc)) not in vis:
                    if nei == tuple(target):
                        return True
                    vis.add(nei)
                    if len(vis) > blockThr:
                        return True
                    q.append(nei)
        return False

    return bfs(*source, target) and bfs(*target, source)


f([], [0, 1], [20, 9])
# f([[0, 1], [1, 0]], [0, 0], [0, 2])
f([[10, 9], [9, 10], [10, 11], [11, 10]], [0, 0], [10, 10])


False

In [13]:
from collections import deque
from pprint import pprint


def f(blocked: list, source: list, target: list):
    if len(blocked) < 2:  # 1或0个阻碍很容易突破
        return True
    # 离散化 将图缩小
    rows = sorted(set(i[0] for i in blocked + [source] + [target]))  # 排序且去重
    cols = sorted(set(i[1] for i in blocked + [source] + [target]))
    r_map, c_map = {}, {}  # 创建原值和离散后的映射，以便将原值映射到新图
    # 对横坐标离散化
    r = 0 if rows[0] == 0 else 1  # r表示索引 初始状态，如果初始贴边，离散也贴边，初始不贴边，离散就在离边最近的1位置
    r_map[rows[0]] = r
    for i in range(1, len(rows)):
        step = 1 if rows[i] == rows[i - 1] + 1 else 2  # 和前面比较，相同的已经去重去过了，如果隔壁(差1)那就离散也隔壁(差1)，否则差2
        r += step  # 往前走步数
        r_map[rows[i]] = r
    sz_row = r + 1 if rows[-1] == 10**6 - 1 else r + 2  # 新图大小，如果最大值贴边(索引10**6 - 1)，那么新图也和离散最大值贴边，那么新图大小r+1，否则r+2
    # 同理对纵坐标离散化
    c = 0 if cols[0] == 0 else 1
    c_map[cols[0]] = c
    for i in range(1, len(cols)):
        step = 1 if cols[i] == cols[i - 1] + 1 else 2
        c += step
        c_map[cols[i]] = c
    sz_col = c + 1 if cols[-1] == 10**6 - 1 else c + 2

    grid = [[0] * sz_col for _ in range(sz_row)]  # 创建新图
    for r, c in blocked + [source]:  # 初始化不能走的位置
        grid[r_map[r]][c_map[c]] = 1
    # pprint(grid)
    sr, sc = r_map[source[0]], c_map[source[1]]  # 初始位置
    er, ec = r_map[target[0]], c_map[target[1]]  # 结束位置
    # 启动广度优先遍历
    queue = deque([(sr, sc)])
    while queue:
        sz = len(queue)
        for _ in range(sz):
            r, c = queue.popleft()
            for nr, nc in ((r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)):
                if 0 <= nr < sz_row and 0 <= nc < sz_col and grid[nr][nc] != 1:
                    if nr == er and nc == ec:  # 一旦找到就推出
                        return True
                    grid[nr][nc] = 1
                    queue.append((nr, nc))
    return False  # 无法走出包围圈


# f([[0,1],[0,2],[1,1],[4,3]],[0,0],[2,2])
f([[0, 999991], [0, 999993], [0, 999996], [1, 999996], [1, 999997], [1, 999998], [1, 999999]], [0, 999997],
  [0, 2])


False

In [15]:
from collections import deque
from pprint import pprint


def f(blocked: list, source: list, target: list):
    if len(blocked) < 2:
        return True
    Max = (len(blocked) + 1) * len(blocked) // 2
    # 离散化
    exist = blocked + [source] + [target]
    rows = sorted(set(i[0] for i in exist))
    cols = sorted(set(i[1] for i in exist))
    r_map, c_map = {}, {}
    r = 0 if rows[0] == 0 else 1
    r_map[rows[0]] = r
    for i in range(1, len(rows)):
        step = 1 if rows[i] == rows[i - 1] + 1 else 2
        r += step
        r_map[rows[i]] = r
    sz_row = r + 1 if rows[-1] == 10**6 - 1 else r + 2

    c = 0 if cols[0] == 0 else 1
    c_map[cols[0]] = c
    for i in range(1, len(cols)):
        step = 1 if cols[i] == cols[i - 1] + 1 else 2
        c += step
        c_map[cols[i]] = c
    sz_col = c + 1 if cols[-1] == 10**6 - 1 else c + 2

    visited = set()
    for r, c in blocked + [source]:
        visited.add((r_map[r], c_map[c]))
    sr, sc = r_map[source[0]], c_map[source[1]]
    er, ec = r_map[target[0]], c_map[target[1]]
    # 广度优先遍历
    queue = deque([(sr, sc)])
    while queue:
        sz = len(queue)
        for _ in range(sz):
            r, c = queue.popleft()
            for nr, nc in ((r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)):
                if 0 <= nr < sz_row and 0 <= nc < sz_col and (nr, nc) not in visited:
                    if nr == er and nc == ec:
                        return True
                    visited.add((nr, nc))
                    queue.append((nr, nc))
    return False


# f([[0,1],[0,2],[1,1],[4,3]],[0,0],[2,2])
f([[0, 999991], [0, 999993], [0, 999996], [1, 999996], [1, 999997], [1, 999998], [1, 999999]], [0, 999997],
  [0, 2])


False

In [22]:
x = set([(1,2)])
tuple([1,2])
if (x:=(1,2)):
    print(x==tuple([1,2]))

True
